Importing open ai gym and the package

In [3]:
import gym
import numpy as np
import airline
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.policies import ActorCriticPolicy
import torch as th
from GruActorCriticPolicy import CustomActorCriticPolicy as GruACP
from LongActorCriticPolicy import CustomActorCriticPolicy as LACP

Define the function for evaluating the model on a given environment for a specified number of episodes. 

In [4]:
def evaluate(model, env, numiters):
    rewards = []
    for j in range(numiters):
        obs = env.reset()
        tot_reward = 0
        for i in range(env.tau):
            action, states = model.predict(obs)
            _, reward, _, _ = env.step(action)
            tot_reward += reward
        rewards.append(tot_reward)
    return np.mean(rewards), np.std(rewards)

# Model is the random policy
def policy(env):
    return env.action_space.sample()
    

Setting up the configuration for the environment

In [23]:
A = np.asarray([[1, 1, 0,0,0,0], [ 0,0, 1, 1, 1, 1], [ 0,0, 0,0, 1, 1] ])
tau = 23
P = np.ones((tau, A.shape[1]))/3
c = [5, 5, 5]
f = range(10, 16)
CONFIG = {'A': A, 'f': f, 'P': P, 'starting_state': c , 'tau': tau}

Making an instance of the environment

In [24]:
np.random.seed(1)
env = gym.make('Airline-v0', config=CONFIG)

In [25]:
check_env(env)

In [21]:
model = A2C(LACP, env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
#Baseline MlpPolicy
model = A2C("MlpPolicy", env, n_steps = 5, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [23]:
#OneSharedLayer
policy_kwargs = dict(net_arch=[64, dict(vf=[64, 64], pi=[64, 64])  ])
model = A2C("MlpPolicy", env,  policy_kwargs = policy_kwargs, n_steps = 5, verbose=1)

array([5, 5, 5])

In [28]:
#WideMlp
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[1024, 1024, 1024 ])
model = A2C("MlpPolicy", env,  policy_kwargs = policy_kwargs, n_steps = 5, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [36]:
#DeepMlp
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch = [32, 32, 64, 64, 128, 256, 128, 64, 64, 32, 32])
model = A2C(LongActorCriticPolicy, env,  policy_kwargs = policy_kwargs, n_steps = 5, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [26]:
#Gru
model = A2C(GruActorCriticPolicy, env, n_steps = 5, verbose=1)

NameError: name 'GruActorCriticPolicy' is not defined

In [27]:
model.learn(total_timesteps =2000)

------------------------------------
| time/                 |          |
|    fps                | 383      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -4.14    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 2.87e-08 |
|    value_loss         | 1.06e-17 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 393      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -4.14    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | 3.3e-08  |
|    value_loss         | 1.33e-17 |
-

In [38]:
model.save("a2cDeep")

In [15]:
obs = env.reset()

In [ ]:
model.predict(obs)

Testing the variables

In [ ]:
print(env.state)
print(env.action_space)
print(env.observation_space) # openAI calles states observations :/

Testing the step function

In [ ]:
env.state = [1,0, 0]
print(env.step(np.asarray([1,1, 1, 1, 1, 1]), 5))
print('***')
env.state = [3, 3, 3]
print(env.step([1,1, 1, 1, 1, 1], 5))

test sample uniformly from action space at random

In [ ]:
env.action_space.sample()

Simulating randomized policy

In [ ]:
e = 50
rewards = []
for j in range(e):
    obs = env.reset()
    tot_reward = 0
    for i in range(tau):
        action, states_ = model.predict(obs)
        obs, reward, dones, info = env.step(action)
        tot_reward += reward
    rewards.append(tot_reward)

Test Evaluate Function

In [28]:
print('Here is the mean and standard deviation of the estimate ' + str(evaluate(model, env, 50)))

Here is the mean and standard deviation of the estimate (108.02, 5.247818594425688)
